In [1]:
import os
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.streaming import StreamingContext

In [2]:

app_name = 'val : WordCountOverStream'

In [3]:
# Create a local StreamingContext with two working thread and batch interval of 1 second
sc = SparkContext("local[2]", 'app_name')
ssc = StreamingContext(sc, 1)

In [4]:
# Create a DStream that will connect to hostname:port, like localhost:9999
lines = ssc.socketTextStream("localhost", 9999)
# sudo apt-get install netcat/ncat/nc

In [5]:
lines

In [6]:
lines.pprint()

In [7]:
words = lines.flatMap(lambda line: line.split(" "))

In [8]:
spark = SparkSession(sc)

def sendPartition(rdd):
    from pyspark.sql import Row
    row = Row("message") # Or some other column name
    rdd = rdd.map(row)
    
    if not rdd.isEmpty():
        from pyspark.ml import Pipeline, PipelineModel
        pipeline = PipelineModel.load("/Users/val/Documents/code/spark/models/spam_model4.4")
        raw = rdd.toDF()
        pipeline.transform(raw).show()
        

lines.foreachRDD(sendPartition)

In [9]:
# Count each word in each batch
pairs = words.map(lambda word: (word, 1))
wordCounts = pairs.reduceByKey(lambda x, y: x + y)

# Print the first ten elements of each RDD generated in this DStream to the console
wordCounts.pprint()

In [9]:
ssc.start()             # Start the computation
ssc.awaitTermination()  # Wait for the computation to terminate

-------------------------------------------
Time: 2019-12-27 15:18:24
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:18:25
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:18:26
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:18:27
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:18:28
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:18:29
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:18:30
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:18:31
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:18:32
----------

-------------------------------------------
Time: 2019-12-27 15:18:39
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:18:40
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:18:41
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:18:42
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:18:43
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:18:44
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:18:45
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:18:46
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:18:47
----------

-------------------------------------------
Time: 2019-12-27 15:19:51
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:19:52
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:19:53
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:19:54
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:19:55
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:19:56
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:19:57
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:19:58
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:19:59
----------

-------------------------------------------
Time: 2019-12-27 15:21:03
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:21:04
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:21:05
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:21:06
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:21:07
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:21:08
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:21:09
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:21:10
-------------------------------------------

-------------------------------------------
Time: 2019-12-27 15:21:11
----------

KeyboardInterrupt: 

In [11]:
ssc.stop()

-------------------------------------------
Time: 2019-12-26 11:32:32
-------------------------------------------

-------------------------------------------
Time: 2019-12-26 11:32:32
-------------------------------------------



In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

In [2]:
# Create DataFrame representing the stream of input lines from connection to localhost:9999
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

# Generate running word count
wordCounts = words.groupBy("word").count()

In [3]:
raw = lines.withColumnRenamed("value","message")

In [4]:
raw

DataFrame[message: string]

In [5]:
from pyspark.ml import Pipeline, PipelineModel
pipeline = PipelineModel.load("/Users/val/Documents/code/spark/models/spam_model4.4")
out = pipeline.transform(raw)['rawPrediction','probability', "prediction"]


In [ ]:
 # Start running the query that prints the running counts to the console
query = out \
    .writeStream \
    .format("console") \
    .start()

query.awaitTermination()